In [1]:
print(1)

1


In [5]:
# Example usage
import json
json1 = json.loads('{"name": "John", "age": 30, "city": "New York", "hobbies": ["reading", "swimming"]}')
json2 = json.loads('{"name": "Jon", "age": 31, "city": "New York", "hobbies": ["reading", "running"]}')

from json_eval import compare_json
similarity_score = compare_json(json1, json1)
print(f"Similarity score: {similarity_score:.2f}%")
similarity_score = compare_json(json1, json2)
print(f"Similarity score: {similarity_score:.2f}%")

Similarity score: 100.00%
Similarity score: 78.57%


In [8]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))

similarity_score = compare_json(expected, expected)
print(f"Similarity score: {similarity_score:.2f}%")
similarity_score = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")

Similarity score: 100.00%
Similarity score: 97.08%
